# Readme:

> 1. The script downloads multiple Global Wind Atlas (GWA) files based on the provided API URLs (referred to as resource URLs on the GWA website download page, https://globalwindatlas.info/en/download/gis-files, as also shown in the screenshot below). 
> 2. The downloads are performed simultaneously and support resumable functionality."
> 3. Print progress every 30 seconds.


![gwa_api](./plot/gwa_api.jpg)

In [ ]:
import requests
import os
import time
import threading

# Define the input URLs and output files

URL1 = "https://globalwindatlas.info/api/gis/global/combined-weibull-a/100"
URL2 = "https://globalwindatlas.info/api/gis/global/combined-weibull-k/100"
URL3 = "https://globalwindatlas.info/api/gis/global/power-density/100"
URL4 = "https://globalwindatlas.info/api/gis/global/wind-speed/100"
URL5 = "https://globalwindatlas.info/api/gis/global/air-density/100"

# File name for saving
output1 = "gwa3_250_combinedWeibullA_100m.tif"
output2 = "gwa3_250_combinedWeibullk_100m.tif"
output3 = "gwa3_250_powerDensity_100m.tif"
output4 = "gwa3_250_windSpeed_100m.tif"
output5 = "gwa3_250_airDensity_100m.tif"

FILES_TO_DOWNLOAD = [
    (URL1, output1),
    # (URL2, output2),
    # (URL3, output3),
    # (URL4, output4),
    # (URL5, output5)
    ]

def get_existing_file_size(file):
    return os.path.getsize(file) if os.path.exists(file) else 0

def format_time(seconds):
    return time.strftime('%H:%M:%S', time.gmtime(seconds))

def download_file_with_resume(url, file_path):
    temp_file = file_path + ".part"  # Temporary file for incomplete downloads
    chunk_size = 8192  # 8 KB per chunk
    last_print_time = time.time()  # Time tracking for progress updates
    start_time = time.time()  # Start time for total duration

    downloaded = get_existing_file_size(temp_file)
    headers = {"Range": f"bytes={downloaded}-"} if downloaded > 0 else {}

    response = requests.get(url, headers=headers, stream=True)
    if response.status_code not in [200, 206]:  # 206 = Partial Content
        print(f"❌ Error downloading {file_path}:", response.status_code, response.text)
        return

    with open(temp_file, "ab") as file:
        for chunk in response.iter_content(chunk_size=chunk_size):
            if chunk:
                file.write(chunk)
                downloaded += len(chunk)

                if time.time() - last_print_time >= 30:  # Print progress every 30 seconds
                    elapsed_time = time.time() - start_time
                    print(f"⬇️ Download progress for {file_path}: {int(downloaded/1024/1024):g} MB downloaded | Elapsed time: {format_time(elapsed_time)}")
                    last_print_time = time.time()

    os.rename(temp_file, file_path)
    total_time = time.time() - start_time
    print(f"Download complete: {file_path} | Total time: {format_time(total_time)}")


# Start downloads in parallel
if __name__ == "__main__":
    threads = []
    for url, output_file in FILES_TO_DOWNLOAD:
        thread = threading.Thread(target=download_file_with_resume, args=(url, output_file))
        thread.start()
        threads.append(thread)
    
    for thread in threads:
        thread.join()
    
    print("All downloads completed!")

    print("Store them in the disk named loacl_data")


Exception in thread Thread-6 (download_file_with_resume):
Traceback (most recent call last):
  File "/Users/ctang/anaconda3/envs/GEO_24/lib/python3.11/site-packages/urllib3/connection.py", line 198, in _new_conn
    sock = connection.create_connection(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/ctang/anaconda3/envs/GEO_24/lib/python3.11/site-packages/urllib3/util/connection.py", line 85, in create_connection
    raise err
  File "/Users/ctang/anaconda3/envs/GEO_24/lib/python3.11/site-packages/urllib3/util/connection.py", line 73, in create_connection
Exception in thread Thread-7 (download_file_with_resume):
Traceback (most recent call last):
  File "/Users/ctang/anaconda3/envs/GEO_24/lib/python3.11/site-packages/urllib3/connection.py", line 198, in _new_conn
    sock.connect(sa)
TimeoutError: [Errno 60] Operation timed out

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/Users/ctang/anaconda3/envs/GEO_24/lib

All downloads completed!
